In [1]:
import typing as T  # pylint: disable=unused-import

import numpy as np
import numpy.typing as npt

from pydrake.solvers import (  # pylint: disable=import-error, no-name-in-module, unused-import
    MathematicalProgram,
    MathematicalProgramResult,
    Solve,
    MosekSolver,
    MosekSolverDetails,
    SolverOptions,
    CommonSolverOption,
)
from pydrake.geometry.optimization import (  # pylint: disable=import-error, no-name-in-module
    HPolyhedron,
    Point,
    ConvexSet,
)

from pydrake.symbolic import ( # pylint: disable=import-error, no-name-in-module, unused-import
    Polynomial,
    Variable,
    Variables,
    Expression,
)  
from program_options import ProgramOptions, FREE_POLY, PSD_POLY

from util import (
    timeit,
    diditwork,
    INFO,
    YAY,
    WARN,
    ERROR,
)  # pylint: disable=import-error, no-name-in-module, unused-import

import plotly.graph_objects as go  # pylint: disable=import-error
from plotly.express.colors import sample_colorscale  # pylint: disable=import-error
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from polynomial_gcs_dual import PolynomialDualGCS, Vertex, Edge
from kink_example import random_uniform_graph_generator, build_m_step_horizon_from_layers, plot_m_step_horizon_from_layers, plot_a_layered_graph, plot_policy_rollout

In [19]:
options = ProgramOptions()
gcs, layers = random_uniform_graph_generator(num_layers=10, options=options, random_seed=2)
plot_a_layered_graph(layers)

Solve took 0.966s
solve successful!
-90.39251855536816
SolutionResult.kSolutionFound
Solver is Mosek
time 0.9208378791809082
rescode 0
solution_status 1


In [20]:
v_0 = layers[0][0]

def add_rollout_plots(fig, gcs, layers, lookahead, vertex, layer_index, color, dx=0.1):
    x,y = plot_m_step_horizon_from_layers(gcs, layers, lookahead, vertex, layer_index,dx=dx)
    x_r, y_r = plot_policy_rollout(gcs, layers, lookahead, vertex, layer_index,dx=dx)
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name=str(lookahead)+"-step-lookahead lb", line=dict(color=color) ))
    fig.add_trace(go.Scatter(x=x_r, y=y_r, mode='lines', name=str(lookahead)+"-step-lookahead ub", line=dict(color=color) ))


In [21]:
start_vertex = layers[0][0]
dx = 0.1

fig = gcs.make_plots(start_vertex.name, dx=dx)
add_rollout_plots(fig, gcs, layers, 1, v_0, 0, "red", dx=dx)
add_rollout_plots(fig, gcs, layers, 2, v_0, 0, "orange", dx=dx)
add_rollout_plots(fig, gcs, layers, 3, v_0, 0, "purple", dx=dx)

In [23]:
fig.update_layout(height=1200, width=1200)
# Set x-axis and y-axis limits
# fig.update_xaxes(range=[1, 5])  # Set x-axis limits
fig.update_yaxes(range=[8.95, 10.25])   # Set y-axis limits

In [2]:
options = ProgramOptions()
options.use_convex_relaxation = False

graph = PolynomialDualGCS(options)
# test out on something simpel

quad_cost = lambda x, y: np.sum([(x[i]-y[i])**2 for i in range(len(x))])

a1 = graph.AddTargetVertex("a1", Point([0]), lambda x: 0 )
b1 = graph.AddVertex("b1", HPolyhedron.MakeBox([-3],[-1]))
b2 = graph.AddVertex("b2", HPolyhedron.MakeBox([1],[3]))
c1 = graph.AddVertex("c1", HPolyhedron.MakeBox([-3],[3]))

graph.AddEdge(c1, b1, quad_cost)
graph.AddEdge(c1, b2, quad_cost)
graph.AddEdge(b1, a1, quad_cost)
graph.AddEdge(b2, a1, quad_cost)

for v in graph.gcs.Edges():
    print(v.name())

graph.solve_for_true_shortest_path("c1", [2])
graph.solve_for_true_shortest_path("c1", [3])


e0
c1 b1
c1 b2
b1 a1
b2 a1


TypeError: PolynomialDualGCS.solve_for_true_shortest_path() missing 1 required positional argument: 'point'

: 

In [7]:
for v in graph.gcs.Edges():
    print(v.name())

e5
c1 b1
c1 b2
b1 a1
b2 a1
e5
e6
e7
e8
